In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys, os
sys.path.append(os.path.abspath("../.."))
from configs import GOOGLE_APPLICATION_CREDENTIALS,GCS_BUCKET_NAME,GCS_PROJECT_ID
from google.cloud import bigquery
from src.utils.io_utils import upload_to_bigquery
from clean_utils import *

In [2]:
client = bigquery.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
table_id = f"{GCS_PROJECT_ID}.{GCS_BUCKET_NAME}.data_cleaned"

In [3]:
query = """SELECT *
FROM `khangtestdbt.xecupredict.bonbanh_com` """
data_bonbanh = client.query(query).to_dataframe()
data_bonbanh.head(1)

,Name,Năm_sản_xuất,Tình_trạng,Số_Km_đã_đi,Xuất_xứ,Kiểu_dáng,Động_cơ,Màu_ngoại_thất,Màu_nội_thất
0,Xe \n\t\t\t\n\t\t\t\t\t\t\tVolvo S90 \n\t\t\t\...,2025,Xe mới,None,Nhập khẩu,Sedan,Xăng 2.0 L,Trắng,Nâu


In [4]:
df_bonbanh = data_bonbanh.copy()

In [5]:
df_bonbanh.columns

Index(['Name', 'Năm_sản_xuất', 'Tình_trạng', 'Số_Km_đã_đi', 'Xuất_xứ',
       'Kiểu_dáng', 'Động_cơ', 'Màu_ngoại_thất', 'Màu_nội_thất'],
      dtype='object')

In [6]:
df_bonbanh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989 entries, 0 to 9988
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Name            9989 non-null   object
 1   Năm_sản_xuất    9989 non-null   object
 2   Tình_trạng      9989 non-null   object
 3   Số_Km_đã_đi     7781 non-null   object
 4   Xuất_xứ         9989 non-null   object
 5   Kiểu_dáng       9989 non-null   object
 6   Động_cơ         9989 non-null   object
 7   Màu_ngoại_thất  9989 non-null   object
 8   Màu_nội_thất    9989 non-null   object
dtypes: object(9)
memory usage: 702.5+ KB


In [7]:
df_bonbanh.isna().sum()

Name                 0
Năm_sản_xuất         0
Tình_trạng           0
Số_Km_đã_đi       2208
Xuất_xứ              0
Kiểu_dáng            0
Động_cơ              0
Màu_ngoại_thất       0
Màu_nội_thất         0
dtype: int64

In [8]:
df_bonbanh.describe(include='all').T

,count,unique,top,freq
Name,9989,7323,Xe \n\t\t\t\n\t\t\t\t\t\t\tToyota Land Cruiser...,95
Năm_sản_xuất,9989,32,2025,2338
Tình_trạng,9989,2,Xe đã dùng,7781
Số_Km_đã_đi,7781,670,0 Km,612
Xuất_xứ,9989,2,Nhập khẩu,5002
Kiểu_dáng,9989,10,SUV,4917
Động_cơ,9989,88,Xăng 2.0 L,2251
Màu_ngoại_thất,9989,18,Trắng,3623
Màu_nội_thất,9989,18,Đen,4931


1.name

In [9]:
df_bonbanh["name"] = df_bonbanh["Name"].apply(clean_name)
df_bonbanh.drop(columns=["Name"], inplace=True)
df_bonbanh.head(1)

,Năm_sản_xuất,Tình_trạng,Số_Km_đã_đi,Xuất_xứ,Kiểu_dáng,Động_cơ,Màu_ngoại_thất,Màu_nội_thất,name
0,2025,Xe mới,None,Nhập khẩu,Sedan,Xăng 2.0 L,Trắng,Nâu,Volvo S90 Ultra B6 AWD 2025 2 Tỷ 69 Triệu


2.price

In [10]:
def extract_price(name_str):
    if not isinstance(name_str, str):
        return None

    name_str = name_str.strip()
    
    match = re.search(r'(\d+(\.\d+)?\s*t[ỷi](?:\s*\d+\s*triệu)?)|(\d+(\.\d+)?\s*triệu)', name_str, re.IGNORECASE)
    if match:
        return match.group(0)
    return None

In [11]:
df_bonbanh['price'] = df_bonbanh['name'].apply(extract_price)
df_bonbanh.head(5)

,Năm_sản_xuất,Tình_trạng,Số_Km_đã_đi,Xuất_xứ,Kiểu_dáng,Động_cơ,Màu_ngoại_thất,Màu_nội_thất,name,price
0,2025,Xe mới,None,Nhập khẩu,Sedan,Xăng 2.0 L,Trắng,Nâu,Volvo S90 Ultra B6 AWD 2025 2 Tỷ 69 Triệu,2 Tỷ 69 Triệu
1,2025,Xe mới,None,Nhập khẩu,Sedan,Xăng 2.0 L,Đen,Nâu,Volvo S90 Ultra B6 AWD 2025 2 Tỷ 132 Triệu,2 Tỷ 132 Triệu
2,2025,Xe đã dùng,279 Km,Nhập khẩu,Sedan,Xăng 2.0 L,Trắng,Nâu,Volvo S90 Ultra B6 AWD 2025 2 Tỷ 99 Triệu,2 Tỷ 99 Triệu
3,2025,Xe đã dùng,"4,000 Km",Nhập khẩu,Sedan,Xăng 2.0 L,Trắng,Nâu,Volvo S90 Ultra B6 AWD 2025 2 Tỷ 39 Triệu,2 Tỷ 39 Triệu
4,2019,Xe đã dùng,"120,000 Km",Nhập khẩu,SUV,Xăng 2.0 L,Đỏ,Đen,Volvo XC40 T5 AWD R Design 2019 1 Tỷ 55 Triệu,1 Tỷ 55 Triệu


In [12]:
def clean_price(x):
    if pd.isna(x):
        return None
    
    x = str(x).lower().strip()

    if x in ["", "none", "nan", "giá liên hệ", "contact", "-"]:
        return None

    x = x.replace("triệu", "triệu").replace("trieuj", "triệu")
    x = x.replace("ty", "tỷ").replace("ti", "tỷ")

    x = x.replace("tỉ", "tỷ")

    x = re.sub(r"\s+", " ", x)

    if "triệu" not in x and "tỷ" not in x:
        digits = re.sub(r"[^\d]", "", x)
        return int(digits) if digits.isdigit() else None

    x = x.replace(",", ".")  

    match = re.match(r"(\d*\.?\d*)\s*tỷ\s*(\d*\.?\d*)?\s*triệu?", x)
    if match:
        ty = float(match.group(1)) if match.group(1) else 0
        trieu = float(match.group(2)) if match.group(2) else 0
        return int(ty * 1_000_000_000 + trieu * 1_000_000)

    if "tỷ" in x:
        num = re.findall(r"[\d\.]+", x)
        if num:
            return int(float(num[0]) * 1_000_000_000)
        return None

    if "triệu" in x:
        num = re.findall(r"[\d\.]+", x)
        if num:
            return int(float(num[0]) * 1_000_000)
        return None

    return None


In [13]:
df_bonbanh["price"] = df_bonbanh["price"].apply(clean_price)
df_bonbanh.sample(5)

,Năm_sản_xuất,Tình_trạng,Số_Km_đã_đi,Xuất_xứ,Kiểu_dáng,Động_cơ,Màu_ngoại_thất,Màu_nội_thất,name,price
4968,2024,Xe đã dùng,"30,000 Km",Lắp ráp trong nước,SUV,Xăng 2.5 L,Trắng,Nâu,Hyundai SantaFe Prestige 2 5L HTRAC 2024 1 Tỷ ...,1099000000
4768,2023,Xe đã dùng,"29,000 Km",Lắp ráp trong nước,Crossover,Xăng 1.6 L,Trắng,Đen,Hyundai Tucson 1 6 AT Turbo HTRAC Đặc biệt 202...,909000000
1909,2022,Xe đã dùng,"50,000 Km",Nhập khẩu,SUV,Dầu 2.0 L,Trắng,Đen,Ford Everest Titanium Plus 2 0L 4x4 AT 2022 1 ...,1265000000
3637,2021,Xe đã dùng,"20,000 Km",Nhập khẩu,Convertible/Cabriolet,Xăng 2.0 L,Trắng,Đỏ,BMW Z4 sDrive30i M Sport 2021 2 Tỷ 299 Triệu,2299000000
9943,2025,Xe mới,None,Nhập khẩu,Sedan,Xăng 1.5 L,Đỏ,-,MG 5 Standard 1 5 AT 2025 395 Triệu,395000000


In [14]:
df_bonbanh["price"].isna().sum()

np.int64(0)

In [15]:
df_bonbanh = df_bonbanh[df_bonbanh["price"] > 50000000]

3.brand

In [16]:
df_bonbanh["brand"] = df_bonbanh["name"].str.strip().str.split().str[0]
df_bonbanh["brand"] = df_bonbanh["brand"].str.lower()

In [17]:
df_bonbanh["brand"].unique()

array(['volvo', 'wuling', 'vinfast', 'volkswagen', 'landrover', 'kia',
       'lamborghini', 'lexus', 'ford', 'honda', 'gac', 'gaz', 'geely',
       'haima', 'haval', 'hongqi', 'hummer', 'hyundai', 'bmw',
       'chevrolet', 'chrysler', 'citroen', 'daewoo', 'daihatsu', 'dodge',
       'dongben', 'ferrari', 'audi', 'baic', 'bentley', 'bestune', 'byd',
       'cadillac', 'chenglong', 'acura', 'aston', 'infiniti', 'isuzu',
       'jaecoo', 'jaguar', 'jeep', 'kenbo', 'mazda', 'lynk', 'maserati',
       'mercedes', 'mclaren', 'toyota', 'triumph', 'subaru', 'suzuki',
       'sym', 'tank', 'thaco', 'mitsubishi', 'porsche', 'peugeot',
       'nissan', 'omoda', 'ram', 'renault', 'rolls', 'samsung', 'skoda',
       'smart', 'ssangyong', 'morgan', 'mg', 'mini'], dtype=object)

4.year

In [18]:
df_bonbanh["Năm_sản_xuất"].sample(5)

7991    2025
5953    2021
9670    2024
6314    2022
4003    2022
Name: Năm_sản_xuất, dtype: object

In [19]:
df_bonbanh["Năm_sản_xuất"].isna().sum()

np.int64(0)

In [20]:
df_bonbanh["age"] = 2025 - df_bonbanh["Năm_sản_xuất"].astype(int)
df_bonbanh.drop(columns=["Năm_sản_xuất"], inplace=True)
df_bonbanh["age"].sample(5)

7926    3
7552    1
4289    0
9618    6
9339    9
Name: age, dtype: int64

5.status

In [21]:
df_bonbanh["Tình_trạng"].unique()

array(['Xe mới', 'Xe đã dùng'], dtype=object)

In [22]:
df_bonbanh = df_bonbanh[df_bonbanh["Tình_trạng"] == "Xe đã dùng"]
df_bonbanh.drop(columns=["Tình_trạng"], inplace=True)

6.km

In [23]:
df_bonbanh["Số_Km_đã_đi"].sample(5)

6649    35,000 Km
4085    13,000 Km
7423    22,000 Km
3466    43,000 Km
1198    10,500 Km
Name: Số_Km_đã_đi, dtype: object

In [24]:
df_bonbanh["Số_Km_đã_đi"].isna().sum()

np.int64(0)

In [25]:
df_bonbanh.rename(columns={"Số_Km_đã_đi": "km"}, inplace=True)

In [26]:
def clean_km(x):
    if pd.isna(x):
        return None
    
    x = str(x).lower().strip()

    x = x.replace("km", "").replace(",", "").strip()

    if x in ["", "none", "nan", "-"]:
        return None

    digits = re.sub(r"[^\d]", "", x)
    return int(digits) if digits.isdigit() else None

In [27]:
df_bonbanh["km"] = df_bonbanh["km"].apply(clean_km)

In [28]:
df_bonbanh["km"].sample(5)

5091     80000
9886     34000
8555    120000
6869    130000
7869     65000
Name: km, dtype: int64

In [29]:
df_bonbanh = df_bonbanh[df_bonbanh["km"] > 100]

In [30]:
df_bonbanh["km"].min()

np.int64(101)

7.origin

In [31]:
df_bonbanh["Xuất_xứ"].unique()

array(['Nhập khẩu', 'Lắp ráp trong nước'], dtype=object)

In [32]:
df_bonbanh = df_bonbanh.rename(columns={"Xuất_xứ": "origin"})

In [33]:
df_bonbanh["origin"] = df_bonbanh["origin"].replace({
    "Lắp ráp trong nước": "trong nước",
    "Nhập khẩu": "nhập khẩu"
})


In [34]:
df_bonbanh["origin"].isna().sum()

np.int64(0)

In [35]:
df_bonbanh["origin"].value_counts()

origin
trong nước    3637
nhập khẩu     3440
Name: count, dtype: int64

8.body

In [36]:
df_bonbanh["Kiểu_dáng"].unique()

array(['Sedan', 'SUV', 'Van/Minivan', 'Coupe', 'Hatchback', 'Crossover',
       'Convertible/Cabriolet', 'Bán tải / Pickup', 'Truck'],
      dtype=object)

In [37]:
df_bonbanh = df_bonbanh.rename(columns={"Kiểu_dáng": "body"})
df_bonbanh["body"] = df_bonbanh["body"].str.lower().str.strip()

In [38]:
df_bonbanh["body"].isna().sum()

np.int64(0)

In [39]:
df_bonbanh["body"].value_counts()

body
suv                      3344
sedan                    1953
crossover                 587
van/minivan               383
hatchback                 379
bán tải / pickup        329
coupe                      52
convertible/cabriolet      43
truck                       7
Name: count, dtype: int64

9.fuel

In [40]:
df_bonbanh["Động_cơ"].unique()

array(['Xăng  2.0 L', 'Hybrid  2.0 L', 'Điện', 'Điện  0.2 L',
       'Xăng  3.6 L', 'Xăng  1.4 L', 'Xăng  2.5 L', 'Xăng  1.8 L',
       'Xăng  1.6 L', 'Xăng  1.0 L', 'Xăng  5.0 L', 'Hybrid  3.0 L',
       'Xăng  3.0 L', 'Dầu  2.0 L', 'Xăng  6.5 L', 'Xăng  5.2 L',
       'Dầu  3.0 L', 'Xăng  3.5 L', 'Xăng  3.4 L', 'Xăng  2.4 L',
       'Hybrid  1.8 L', 'Hybrid  2.5 L', 'Xăng  4.6 L', 'Xăng  4.7 L',
       'Hybrid  2.4 L', 'Hybrid  3.5 L', 'Hybrid  5.0 L', 'Xăng  5.7 L',
       'Xăng  1.5 L', 'Dầu  2.2 L', 'Hybrid  1.6 L', 'Xăng  2.3 L',
       'Dầu  2.5 L', 'Dầu  3.2 L', 'Dầu  2.4 L', 'Dầu  2.3 L',
       'Dầu  2.8 L', 'Xăng  1.2 L', 'Xăng  6.0 L', 'Dầu', 'Dầu  3.9 L',
       'Xăng  1.1 L', 'Xăng', 'Xăng  4.8 L', 'Xăng  1.3 L', 'Xăng  3.9 L',
       'Xăng  2.6 L', 'Xăng  4.2 L', 'Xăng  4.0 L', 'Xăng  6.8 L',
       'Xăng  4.4 L', 'Hybrid  1.5 L', 'Xăng  6.2 L', 'Xăng  3.7 L',
       'Xăng  5.9 L', 'Xăng  1.25 L', 'Xăng  3.3 L', 'Dầu  1.9 L',
       'Dầu  1.5 L', 'Xăng  2.7 L', 'Điện  4.

In [41]:
df_bonbanh = df_bonbanh.rename(columns={"Động_cơ": "fuel"})
df_bonbanh["fuel"] = df_bonbanh["fuel"].apply(clean_fuel)

In [42]:
df_bonbanh["fuel"].isna().sum()

np.int64(0)

In [43]:
df_bonbanh["fuel"].unique()

array(['Xăng', 'Hybrid', 'Điện', 'Dầu'], dtype=object)

In [44]:
df_bonbanh["fuel"].value_counts()

fuel
Xăng      5608
Dầu        984
Điện       270
Hybrid     215
Name: count, dtype: int64

In [45]:
df_bonbanh = df_bonbanh[df_bonbanh["fuel"] != "Khác"]

In [46]:
df_bonbanh["fuel"].unique()

array(['Xăng', 'Hybrid', 'Điện', 'Dầu'], dtype=object)

10.dropcol

In [47]:
df_bonbanh.drop(columns=["Màu_ngoại_thất", "Màu_nội_thất"], inplace=True)

In [48]:
df_bonbanh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7077 entries, 2 to 9988
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   km      7077 non-null   int64 
 1   origin  7077 non-null   object
 2   body    7077 non-null   object
 3   fuel    7077 non-null   object
 4   name    7077 non-null   object
 5   price   7077 non-null   int64 
 6   brand   7077 non-null   object
 7   age     7077 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 497.6+ KB


11.updata

In [49]:
df_bonbanh.sample(10)

,km,origin,body,fuel,name,price,brand,age
1442,10000,trong nước,crossover,Xăng,Kia Seltos Premium 1 5 AT 2024 709 Triệu,709000000,kia,1
5795,4000,trong nước,sedan,Xăng,Mercedes Benz C class C200 Avantgarde Plus 202...,1479000000,mercedes,2
2311,2000,trong nước,suv,Xăng,Ford Territory Titanium X 1 5 AT 2025 935 Triệu,935000000,ford,0
1157,2300,nhập khẩu,van/minivan,Hybrid,Lexus LM 500h 2025 8 Tỷ 900 Triệu,8900000000,lexus,0
1328,180000,trong nước,suv,Dầu,Kia Sorento DATH 2 2L 2WD 2014 450 Triệu,450000000,kia,11
7252,50000,nhập khẩu,suv,Xăng,Toyota Corolla Cross 1 8G 2023 680 Triệu,680000000,toyota,2
2416,130000,trong nước,van/minivan,Dầu,Ford Transit Van 2016 330 Triệu,330000000,ford,9
9797,38000,nhập khẩu,suv,Xăng,Mercedes Benz GLS 450 4Matic 2022 4 Tỷ 139 Triệu,4139000000,mercedes,3
5060,6000,nhập khẩu,crossover,Xăng,Mazda CX3 Luxury 1 5 AT 2025 619 Triệu,619000000,mazda,0
8586,60000,nhập khẩu,suv,Xăng,Porsche Macan 2 0 2017 1 Tỷ 680 Triệu,1680000000,porsche,8


In [50]:
upload_to_bigquery(df_bonbanh, table_id=table_id, if_exists="replace" )

✅ Uploaded 7077 rows to khangtestdbt.xecupredict.data_cleaned


In [51]:
# end